In [7]:
import urllib.request
import urllib.parse
import json
import re
import datetime
from collections import namedtuple
from sys import intern
import pytz
import sqltables
import plotly.graph_objs as go
from tqdm.notebook import tqdm

In [2]:
import sys
sys.path.append("..")
import energydata.esett
import energydata.energinet
import energydata.nordpoolspot

In [16]:
import importlib
energydata.esett = importlib.reload(energydata.esett)

In [3]:
db = sqltables.Database()

def create_table_from_dicts(db, dicts):
    column_types = {}
    for k, v in dicts[0].items():
        column_types[k] = "float" if type(v) in {float, int} else "text"
    column_names = list(column_types.keys())
    rows = []
    for d in dicts:
        row = [d[k] for k in column_names]
        rows.append(row)
    return db.create_table(rows=rows, column_names=column_names, column_types=column_types)

In [4]:
start_datetime = "2022-04-01T00:00:00"
end_datetime = "2023-03-31T23:00:00"
price_areas = ["DK1", "DK2"]

In [19]:
esett_datasets = []

for price_area in price_areas:
    esett_descriptor = {
        "start_datetime": start_datetime,
        "end_datetime": end_datetime,
        "MBA": price_area,
        "dataset": "Prices"
    }

    esett_dataset = energydata.esett.ESett(esett_descriptor)
    esett_datasets.append(esett_dataset)

esett_data = [row for dataset in esett_datasets for row in dataset.fetch_data()]

balancing_price_data = create_table_from_dicts(db, esett_data)
balancing_prices = balancing_price_data.view("""
select datetime_start, datetime_end, "imblSalesPrice" as balancing_price, mba as price_area, "mainDirRegPowerPerMBA" as imbalance_direction from _
""")
balancing_prices

|datetime\_start|datetime\_end|balancing\_price|price\_area|imbalance\_direction|
|-|-|-|-|-|
|\'2022\-04\-01T00\:00\:00\+02\:00\'|\'2022\-04\-01T01\:00\:00\+02\:00\'|173\.26|\'DK1\'|0\.0|
|\'2022\-04\-01T01\:00\:00\+02\:00\'|\'2022\-04\-01T02\:00\:00\+02\:00\'|152\.3|\'DK1\'|0\.0|
|\'2022\-04\-01T02\:00\:00\+02\:00\'|\'2022\-04\-01T03\:00\:00\+02\:00\'|195\.0|\'DK1\'|1\.0|
|\'2022\-04\-01T03\:00\:00\+02\:00\'|\'2022\-04\-01T04\:00\:00\+02\:00\'|188\.19|\'DK1\'|1\.0|
|\'2022\-04\-01T04\:00\:00\+02\:00\'|\'2022\-04\-01T05\:00\:00\+02\:00\'|160\.0|\'DK1\'|1\.0|
|\'2022\-04\-01T05\:00\:00\+02\:00\'|\'2022\-04\-01T06\:00\:00\+02\:00\'|194\.0|\'DK1\'|1\.0|
|\'2022\-04\-01T06\:00\:00\+02\:00\'|\'2022\-04\-01T07\:00\:00\+02\:00\'|194\.0|\'DK1\'|1\.0|
|\'2022\-04\-01T07\:00\:00\+02\:00\'|\'2022\-04\-01T08\:00\:00\+02\:00\'|217\.0|\'DK1\'|1\.0|
|\'2022\-04\-01T08\:00\:00\+02\:00\'|\'2022\-04\-01T09\:00\:00\+02\:00\'|250\.0|\'DK1\'|1\.0|
|\'2022\-04\-01T09\:00\:00\+02\:00\'|\'2022\-04\-01T10\:00\:00\+02\:00\'|210\.5|\'DK1\'|1\.0|
|\'2022\-04\-01T10\:00\:00\+02\:00\'|\'2022\-04\-01T11\:00\:00\+02\:00\'|181\.0|\'DK1\'|\-1\.0|
|\'2022\-04\-01T11\:00\:00\+02\:00\'|\'2022\-04\-01T12\:00\:00\+02\:00\'|181\.14|\'DK1\'|0\.0|
|\'2022\-04\-01T12\:00\:00\+02\:00\'|\'2022\-04\-01T13\:00\:00\+02\:00\'|192\.5|\'DK1\'|1\.0|
|\'2022\-04\-01T13\:00\:00\+02\:00\'|\'2022\-04\-01T14\:00\:00\+02\:00\'|182\.53|\'DK1\'|1\.0|
|\'2022\-04\-01T14\:00\:00\+02\:00\'|\'2022\-04\-01T15\:00\:00\+02\:00\'|192\.5|\'DK1\'|1\.0|
|\'2022\-04\-01T15\:00\:00\+02\:00\'|\'2022\-04\-01T16\:00\:00\+02\:00\'|85\.0|\'DK1\'|\-1\.0|
|...|...|...|...|...|


In [20]:
balancing_price_data.view("""
select * from _ order by datetime_start
""")

|timestamp|mba|imblSalesPrice|imblPurchasePrice|upRegPrice|downRegPrice|mainDirRegPowerPerMBA|valueOfAvoidedActivation|incentivisingComponent|datetime\_start|datetime\_end|
|-|-|-|-|-|-|-|-|-|-|-|
|\'2022\-04\-01T00\:00\:00\'|\'DK1\'|173\.26|173\.26|173\.26|173\.26|0\.0|None|None|\'2022\-04\-01T00\:00\:00\+02\:00\'|\'2022\-04\-01T01\:00\:00\+02\:00\'|
|\'2022\-04\-01T00\:00\:00\'|\'DK2\'|150\.0|150\.0|150\.0|111\.72|1\.0|None|None|\'2022\-04\-01T00\:00\:00\+02\:00\'|\'2022\-04\-01T01\:00\:00\+02\:00\'|
|\'2022\-04\-01T01\:00\:00\'|\'DK1\'|152\.3|152\.3|152\.3|152\.3|0\.0|\'152\.2\'|\'0\.1\'|\'2022\-04\-01T01\:00\:00\+02\:00\'|\'2022\-04\-01T02\:00\:00\+02\:00\'|
|\'2022\-04\-01T01\:00\:00\'|\'DK2\'|125\.0|125\.0|125\.0|99\.64|1\.0|None|None|\'2022\-04\-01T01\:00\:00\+02\:00\'|\'2022\-04\-01T02\:00\:00\+02\:00\'|
|\'2022\-04\-01T02\:00\:00\'|\'DK1\'|195\.0|195\.0|195\.0|159\.9|1\.0|None|None|\'2022\-04\-01T02\:00\:00\+02\:00\'|\'2022\-04\-01T03\:00\:00\+02\:00\'|
|\'2022\-04\-01T02\:00\:00\'|\'DK2\'|195\.0|195\.0|195\.0|94\.48|1\.0|None|None|\'2022\-04\-01T02\:00\:00\+02\:00\'|\'2022\-04\-01T03\:00\:00\+02\:00\'|
|\'2022\-04\-01T03\:00\:00\'|\'DK1\'|188\.19|188\.19|188\.19|151\.0|1\.0|None|None|\'2022\-04\-01T03\:00\:00\+02\:00\'|\'2022\-04\-01T04\:00\:00\+02\:00\'|
|\'2022\-04\-01T03\:00\:00\'|\'DK2\'|188\.19|188\.19|188\.19|99\.59|1\.0|None|None|\'2022\-04\-01T03\:00\:00\+02\:00\'|\'2022\-04\-01T04\:00\:00\+02\:00\'|
|\'2022\-04\-01T04\:00\:00\'|\'DK1\'|160\.0|160\.0|160\.0|150\.0|1\.0|None|None|\'2022\-04\-01T04\:00\:00\+02\:00\'|\'2022\-04\-01T05\:00\:00\+02\:00\'|
|\'2022\-04\-01T04\:00\:00\'|\'DK2\'|160\.0|160\.0|160\.0|109\.02|1\.0|None|None|\'2022\-04\-01T04\:00\:00\+02\:00\'|\'2022\-04\-01T05\:00\:00\+02\:00\'|
|\'2022\-04\-01T05\:00\:00\'|\'DK1\'|194\.0|194\.0|194\.0|170\.0|1\.0|None|None|\'2022\-04\-01T05\:00\:00\+02\:00\'|\'2022\-04\-01T06\:00\:00\+02\:00\'|
|\'2022\-04\-01T05\:00\:00\'|\'DK2\'|194\.0|194\.0|194\.0|137\.22|1\.0|None|None|\'2022\-04\-01T05\:00\:00\+02\:00\'|\'2022\-04\-01T06\:00\:00\+02\:00\'|
|\'2022\-04\-01T06\:00\:00\'|\'DK1\'|194\.0|194\.0|194\.0|181\.41|1\.0|None|None|\'2022\-04\-01T06\:00\:00\+02\:00\'|\'2022\-04\-01T07\:00\:00\+02\:00\'|
|\'2022\-04\-01T06\:00\:00\'|\'DK2\'|194\.0|194\.0|194\.0|181\.41|1\.0|None|None|\'2022\-04\-01T06\:00\:00\+02\:00\'|\'2022\-04\-01T07\:00\:00\+02\:00\'|
|\'2022\-04\-01T07\:00\:00\'|\'DK1\'|217\.0|217\.0|217\.0|200\.03|1\.0|None|None|\'2022\-04\-01T07\:00\:00\+02\:00\'|\'2022\-04\-01T08\:00\:00\+02\:00\'|
|\'2022\-04\-01T07\:00\:00\'|\'DK2\'|217\.0|217\.0|217\.0|200\.03|1\.0|None|None|\'2022\-04\-01T07\:00\:00\+02\:00\'|\'2022\-04\-01T08\:00\:00\+02\:00\'|
|...|...|...|...|...|...|...|...|...|...|...|


In [21]:
energinet_descriptor = {
    "start_datetime": start_datetime,
    "end_datetime": end_datetime,
    "dataset": "Elspotprices"
}

energinet_dataset = energydata.energinet.EnergiDataServiceDk(energinet_descriptor)

spot_price_data = create_table_from_dicts(db, list(energinet_dataset.fetch_data()))
spot_prices = spot_price_data.view("""
select datetime_start, datetime_end, "SpotPriceEUR" as spot_price, "PriceArea" as price_area
from _
""")
spot_prices

|datetime\_start|datetime\_end|spot\_price|price\_area|
|-|-|-|-|
|\'2023\-03\-31T22\:00\:00\+02\:00\'|\'2023\-03\-31T23\:00\:00\+02\:00\'|86\.730003|\'DE\'|
|\'2023\-03\-31T22\:00\:00\+02\:00\'|\'2023\-03\-31T23\:00\:00\+02\:00\'|86\.730003|\'DK1\'|
|\'2023\-03\-31T22\:00\:00\+02\:00\'|\'2023\-03\-31T23\:00\:00\+02\:00\'|35\.849998|\'DK2\'|
|\'2023\-03\-31T22\:00\:00\+02\:00\'|\'2023\-03\-31T23\:00\:00\+02\:00\'|89\.43|\'NO2\'|
|\'2023\-03\-31T22\:00\:00\+02\:00\'|\'2023\-03\-31T23\:00\:00\+02\:00\'|35\.849998|\'SE3\'|
|\'2023\-03\-31T22\:00\:00\+02\:00\'|\'2023\-03\-31T23\:00\:00\+02\:00\'|35\.849998|\'SE4\'|
|\'2023\-03\-31T22\:00\:00\+02\:00\'|\'2023\-03\-31T23\:00\:00\+02\:00\'|69\.169998|\'SYSTEM\'|
|\'2023\-03\-31T21\:00\:00\+02\:00\'|\'2023\-03\-31T22\:00\:00\+02\:00\'|98\.830002|\'DE\'|
|\'2023\-03\-31T21\:00\:00\+02\:00\'|\'2023\-03\-31T22\:00\:00\+02\:00\'|98\.150002|\'DK1\'|
|\'2023\-03\-31T21\:00\:00\+02\:00\'|\'2023\-03\-31T22\:00\:00\+02\:00\'|40\.93|\'DK2\'|
|\'2023\-03\-31T21\:00\:00\+02\:00\'|\'2023\-03\-31T22\:00\:00\+02\:00\'|95\.449997|\'NO2\'|
|\'2023\-03\-31T21\:00\:00\+02\:00\'|\'2023\-03\-31T22\:00\:00\+02\:00\'|40\.93|\'SE3\'|
|\'2023\-03\-31T21\:00\:00\+02\:00\'|\'2023\-03\-31T22\:00\:00\+02\:00\'|40\.93|\'SE4\'|
|\'2023\-03\-31T21\:00\:00\+02\:00\'|\'2023\-03\-31T22\:00\:00\+02\:00\'|85\.830002|\'SYSTEM\'|
|\'2023\-03\-31T20\:00\:00\+02\:00\'|\'2023\-03\-31T21\:00\:00\+02\:00\'|109\.980003|\'DE\'|
|\'2023\-03\-31T20\:00\:00\+02\:00\'|\'2023\-03\-31T21\:00\:00\+02\:00\'|101\.849998|\'DK1\'|
|...|...|...|...|


In [8]:
nordpool_datasets = []

for price_area in price_areas:
    nordpool_descriptor = {
        "start_datetime": start_datetime,
        "end_datetime": end_datetime,
        "zone": price_area,
        "dataset": "intraday"
    }

    nordpool_dataset = energydata.nordpoolspot.NordpoolSpot(nordpool_descriptor)
    nordpool_datasets.append(nordpool_dataset)

nordpool_data = list(tqdm(row for dataset in nordpool_datasets for row in dataset.fetch_data()))

intraday_price_data = create_table_from_dicts(db, nordpool_data)
intraday_prices = intraday_price_data.view("""select *, zone as price_area from _""")


0it [00:00, ?it/s]

In [25]:
intraday_prices = intraday_price_data.view("""select *, zone as price_area, start_datetime as datetime_start, end_datetime as datetime_end from _""")
intraday_prices

|start\_datetime|end\_datetime|Product|High|Low|Last|Avg|Volume|zone|price\_area|datetime\_start|datetime\_end|
|-|-|-|-|-|-|-|-|-|-|-|-|
|\'2022\-04\-01T00\:00\:00\+02\:00\'|\'2022\-04\-01T01\:00\:00\+02\:00\'|\'PH\-20220401\-01 \(X\)\'|210\.0|142\.11|209\.83|193\.26|52\.0|\'DK1\'|\'DK1\'|\'2022\-04\-01T00\:00\:00\+02\:00\'|\'2022\-04\-01T01\:00\:00\+02\:00\'|
|\'2022\-04\-01T01\:00\:00\+02\:00\'|\'2022\-04\-01T02\:00\:00\+02\:00\'|\'PH\-20220401\-02 \(X\)\'|210\.85|142\.2|200\.0|194\.58|177\.3|\'DK1\'|\'DK1\'|\'2022\-04\-01T01\:00\:00\+02\:00\'|\'2022\-04\-01T02\:00\:00\+02\:00\'|
|\'2022\-04\-01T02\:00\:00\+02\:00\'|\'2022\-04\-01T03\:00\:00\+02\:00\'|\'PH\-20220401\-03 \(X\)\'|219\.0|139\.0|219\.0|192\.95|170\.1|\'DK1\'|\'DK1\'|\'2022\-04\-01T02\:00\:00\+02\:00\'|\'2022\-04\-01T03\:00\:00\+02\:00\'|
|\'2022\-04\-01T03\:00\:00\+02\:00\'|\'2022\-04\-01T04\:00\:00\+02\:00\'|\'PH\-20220401\-04 \(X\)\'|213\.23|141\.5|180\.0|184\.63|80\.6|\'DK1\'|\'DK1\'|\'2022\-04\-01T03\:00\:00\+02\:00\'|\'2022\-04\-01T04\:00\:00\+02\:00\'|
|\'2022\-04\-01T04\:00\:00\+02\:00\'|\'2022\-04\-01T05\:00\:00\+02\:00\'|\'PH\-20220401\-05 \(X\)\'|261\.1|140\.92|261\.1|203\.41|68\.6|\'DK1\'|\'DK1\'|\'2022\-04\-01T04\:00\:00\+02\:00\'|\'2022\-04\-01T05\:00\:00\+02\:00\'|
|\'2022\-04\-01T05\:00\:00\+02\:00\'|\'2022\-04\-01T06\:00\:00\+02\:00\'|\'PH\-20220401\-06 \(X\)\'|269\.08|155\.0|233\.45|222\.15|272\.6|\'DK1\'|\'DK1\'|\'2022\-04\-01T05\:00\:00\+02\:00\'|\'2022\-04\-01T06\:00\:00\+02\:00\'|
|\'2022\-04\-01T06\:00\:00\+02\:00\'|\'2022\-04\-01T07\:00\:00\+02\:00\'|\'PH\-20220401\-07 \(X\)\'|269\.99|191\.37|269\.98|207\.75|144\.3|\'DK1\'|\'DK1\'|\'2022\-04\-01T06\:00\:00\+02\:00\'|\'2022\-04\-01T07\:00\:00\+02\:00\'|
|\'2022\-04\-01T07\:00\:00\+02\:00\'|\'2022\-04\-01T08\:00\:00\+02\:00\'|\'PH\-20220401\-08 \(X\)\'|232\.5|175\.0|228\.97|205\.19|193\.9|\'DK1\'|\'DK1\'|\'2022\-04\-01T07\:00\:00\+02\:00\'|\'2022\-04\-01T08\:00\:00\+02\:00\'|
|\'2022\-04\-01T08\:00\:00\+02\:00\'|\'2022\-04\-01T09\:00\:00\+02\:00\'|\'PH\-20220401\-09 \(X\)\'|222\.93|209\.0|219\.01|216\.2|353\.5|\'DK1\'|\'DK1\'|\'2022\-04\-01T08\:00\:00\+02\:00\'|\'2022\-04\-01T09\:00\:00\+02\:00\'|
|\'2022\-04\-01T09\:00\:00\+02\:00\'|\'2022\-04\-01T10\:00\:00\+02\:00\'|\'PH\-20220401\-10 \(X\)\'|224\.37|175\.0|216\.69|208\.7|219\.2|\'DK1\'|\'DK1\'|\'2022\-04\-01T09\:00\:00\+02\:00\'|\'2022\-04\-01T10\:00\:00\+02\:00\'|
|\'2022\-04\-01T10\:00\:00\+02\:00\'|\'2022\-04\-01T11\:00\:00\+02\:00\'|\'PH\-20220401\-11 \(X\)\'|223\.59|193\.0|196\.0|210\.02|344\.4|\'DK1\'|\'DK1\'|\'2022\-04\-01T10\:00\:00\+02\:00\'|\'2022\-04\-01T11\:00\:00\+02\:00\'|
|\'2022\-04\-01T11\:00\:00\+02\:00\'|\'2022\-04\-01T12\:00\:00\+02\:00\'|\'PH\-20220401\-12 \(X\)\'|225\.4|190\.81|209\.0|210\.46|520\.3|\'DK1\'|\'DK1\'|\'2022\-04\-01T11\:00\:00\+02\:00\'|\'2022\-04\-01T12\:00\:00\+02\:00\'|
|\'2022\-04\-01T12\:00\:00\+02\:00\'|\'2022\-04\-01T13\:00\:00\+02\:00\'|\'PH\-20220401\-13 \(X\)\'|220\.0|180\.31|190\.5|205\.07|424\.1|\'DK1\'|\'DK1\'|\'2022\-04\-01T12\:00\:00\+02\:00\'|\'2022\-04\-01T13\:00\:00\+02\:00\'|
|\'2022\-04\-01T13\:00\:00\+02\:00\'|\'2022\-04\-01T14\:00\:00\+02\:00\'|\'PH\-20220401\-14 \(X\)\'|204\.69|153\.51|153\.51|180\.53|144\.4|\'DK1\'|\'DK1\'|\'2022\-04\-01T13\:00\:00\+02\:00\'|\'2022\-04\-01T14\:00\:00\+02\:00\'|
|\'2022\-04\-01T14\:00\:00\+02\:00\'|\'2022\-04\-01T15\:00\:00\+02\:00\'|\'PH\-20220401\-15 \(X\)\'|225\.0|160\.11|208\.0|197\.85|284\.6|\'DK1\'|\'DK1\'|\'2022\-04\-01T14\:00\:00\+02\:00\'|\'2022\-04\-01T15\:00\:00\+02\:00\'|
|\'2022\-04\-01T15\:00\:00\+02\:00\'|\'2022\-04\-01T16\:00\:00\+02\:00\'|\'PH\-20220401\-16 \(X\)\'|221\.0|135\.37|179\.0|195\.96|253\.8|\'DK1\'|\'DK1\'|\'2022\-04\-01T15\:00\:00\+02\:00\'|\'2022\-04\-01T16\:00\:00\+02\:00\'|
|...|...|...|...|...|...|...|...|...|...|...|...|


In [26]:
prices = db.query("""
select 
    spot.*, 
    balancing_price, 
    imbalance_direction,
    intraday."Avg" as intraday_price
from spot 
join balancing using (price_area, datetime_start, datetime_end)
join intraday using (price_area, datetime_start, datetime_end)
""", bindings={"balancing": balancing_prices, "spot": spot_prices, "intraday": intraday_prices})
prices.view("""select * from _ order by datetime_start""")

|datetime\_start|datetime\_end|spot\_price|price\_area|balancing\_price|imbalance\_direction|intraday\_price|
|-|-|-|-|-|-|-|
|\'2022\-04\-01T00\:00\:00\+02\:00\'|\'2022\-04\-01T01\:00\:00\+02\:00\'|173\.259995|\'DK1\'|173\.26|0\.0|193\.26|
|\'2022\-04\-01T00\:00\:00\+02\:00\'|\'2022\-04\-01T01\:00\:00\+02\:00\'|111\.720001|\'DK2\'|150\.0|1\.0|122\.54|
|\'2022\-04\-01T01\:00\:00\+02\:00\'|\'2022\-04\-01T02\:00\:00\+02\:00\'|152\.300003|\'DK1\'|152\.3|0\.0|194\.58|
|\'2022\-04\-01T01\:00\:00\+02\:00\'|\'2022\-04\-01T02\:00\:00\+02\:00\'|99\.639999|\'DK2\'|125\.0|1\.0|103\.61|
|\'2022\-04\-01T02\:00\:00\+02\:00\'|\'2022\-04\-01T03\:00\:00\+02\:00\'|159\.899994|\'DK1\'|195\.0|1\.0|192\.95|
|\'2022\-04\-01T02\:00\:00\+02\:00\'|\'2022\-04\-01T03\:00\:00\+02\:00\'|94\.480003|\'DK2\'|195\.0|1\.0|101\.31|
|\'2022\-04\-01T03\:00\:00\+02\:00\'|\'2022\-04\-01T04\:00\:00\+02\:00\'|151\.0|\'DK1\'|188\.19|1\.0|184\.63|
|\'2022\-04\-01T03\:00\:00\+02\:00\'|\'2022\-04\-01T04\:00\:00\+02\:00\'|99\.589996|\'DK2\'|188\.19|1\.0|102\.22|
|\'2022\-04\-01T04\:00\:00\+02\:00\'|\'2022\-04\-01T05\:00\:00\+02\:00\'|150\.0|\'DK1\'|160\.0|1\.0|203\.41|
|\'2022\-04\-01T04\:00\:00\+02\:00\'|\'2022\-04\-01T05\:00\:00\+02\:00\'|109\.019997|\'DK2\'|160\.0|1\.0|109\.09|
|\'2022\-04\-01T05\:00\:00\+02\:00\'|\'2022\-04\-01T06\:00\:00\+02\:00\'|170\.0|\'DK1\'|194\.0|1\.0|222\.15|
|\'2022\-04\-01T05\:00\:00\+02\:00\'|\'2022\-04\-01T06\:00\:00\+02\:00\'|137\.220001|\'DK2\'|194\.0|1\.0|144\.76|
|\'2022\-04\-01T06\:00\:00\+02\:00\'|\'2022\-04\-01T07\:00\:00\+02\:00\'|181\.410004|\'DK1\'|194\.0|1\.0|207\.75|
|\'2022\-04\-01T06\:00\:00\+02\:00\'|\'2022\-04\-01T07\:00\:00\+02\:00\'|181\.410004|\'DK2\'|194\.0|1\.0|213\.35|
|\'2022\-04\-01T07\:00\:00\+02\:00\'|\'2022\-04\-01T08\:00\:00\+02\:00\'|200\.029999|\'DK1\'|217\.0|1\.0|205\.19|
|\'2022\-04\-01T07\:00\:00\+02\:00\'|\'2022\-04\-01T08\:00\:00\+02\:00\'|200\.029999|\'DK2\'|217\.0|1\.0|208\.73|
|...|...|...|...|...|...|...|


In [30]:
def extract_columns(table):
    it = iter(table)
    return dict(zip(it.column_names, zip(*it)))

In [42]:
traces = []
for price_area in price_areas:
    rows = prices.table("""
    select spot_price, intraday_price-spot_price, datetime_start 
    from _
    where price_area = ? and datetime_start like '%T16:00%'
    """, parameters=[price_area])
    [x, y, labels] = zip(*rows)
    traces.append({"x": x, "y": y, "text": labels, "name": price_area, "mode": "markers", "marker": {"opacity": 0.5}})
layout = {
    "xaxis": {"title": "Day-ahead Price"},
    "yaxis": {"title": "Balancing Price"},
    "title": f"From {start_datetime} to {end_datetime}"
}
go.Figure(data=traces, layout=layout).show()

In [100]:
max_prices = prices.view("""select *, max(spot_price, balancing_price) as max_price from _""")

In [101]:
max_prices

|datetime\_start|datetime\_end|spot\_price|price\_area|balancing\_price|imbalance\_direction|max\_price|
|-|-|-|-|-|-|-|
|\'2023\-03\-31T22\:00\:00\+02\:00\'|\'2023\-03\-31T23\:00\:00\+02\:00\'|86\.730003|\'DK1\'|86\.73|0\.0|86\.730003|
|\'2023\-03\-31T22\:00\:00\+02\:00\'|\'2023\-03\-31T23\:00\:00\+02\:00\'|35\.849998|\'DK2\'|85\.0|1\.0|85\.0|
|\'2023\-03\-31T21\:00\:00\+02\:00\'|\'2023\-03\-31T22\:00\:00\+02\:00\'|98\.150002|\'DK1\'|98\.15|0\.0|98\.150002|
|\'2023\-03\-31T21\:00\:00\+02\:00\'|\'2023\-03\-31T22\:00\:00\+02\:00\'|40\.93|\'DK2\'|85\.0|1\.0|85\.0|
|\'2023\-03\-31T20\:00\:00\+02\:00\'|\'2023\-03\-31T21\:00\:00\+02\:00\'|101\.849998|\'DK1\'|101\.85|0\.0|101\.85|
|\'2023\-03\-31T20\:00\:00\+02\:00\'|\'2023\-03\-31T21\:00\:00\+02\:00\'|48\.080002|\'DK2\'|85\.0|1\.0|85\.0|
|\'2023\-03\-31T19\:00\:00\+02\:00\'|\'2023\-03\-31T20\:00\:00\+02\:00\'|110\.459999|\'DK1\'|110\.46|0\.0|110\.46|
|\'2023\-03\-31T19\:00\:00\+02\:00\'|\'2023\-03\-31T20\:00\:00\+02\:00\'|92\.5|\'DK2\'|92\.5|0\.0|92\.5|
|\'2023\-03\-31T18\:00\:00\+02\:00\'|\'2023\-03\-31T19\:00\:00\+02\:00\'|108\.370003|\'DK1\'|174\.99|1\.0|174\.99|
|\'2023\-03\-31T18\:00\:00\+02\:00\'|\'2023\-03\-31T19\:00\:00\+02\:00\'|108\.370003|\'DK2\'|174\.99|1\.0|174\.99|
|\'2023\-03\-31T17\:00\:00\+02\:00\'|\'2023\-03\-31T18\:00\:00\+02\:00\'|103\.290001|\'DK1\'|80\.5|\-1\.0|103\.290001|
|\'2023\-03\-31T17\:00\:00\+02\:00\'|\'2023\-03\-31T18\:00\:00\+02\:00\'|103\.290001|\'DK2\'|80\.5|\-1\.0|103\.290001|
|\'2023\-03\-31T16\:00\:00\+02\:00\'|\'2023\-03\-31T17\:00\:00\+02\:00\'|101\.889999|\'DK1\'|80\.0|\-1\.0|101\.889999|
|\'2023\-03\-31T16\:00\:00\+02\:00\'|\'2023\-03\-31T17\:00\:00\+02\:00\'|101\.889999|\'DK2\'|80\.0|\-1\.0|101\.889999|
|\'2023\-03\-31T15\:00\:00\+02\:00\'|\'2023\-03\-31T16\:00\:00\+02\:00\'|94\.720001|\'DK1\'|80\.0|\-1\.0|94\.720001|
|\'2023\-03\-31T15\:00\:00\+02\:00\'|\'2023\-03\-31T16\:00\:00\+02\:00\'|94\.720001|\'DK2\'|80\.0|\-1\.0|94\.720001|
|...|...|...|...|...|...|...|


In [102]:
[timestamps, spot_price_values, balancing_price_values, max_price_values] = zip(*max_prices.view("""
select datetime_start, spot_price, balancing_price, max_price from _
where datetime_start like '%T16:00%'
"""))
t = {"x": timestamps, "line": {"shape": "hv"}}
traces = [
    {**t, "y": spot_price_values, "name": "Spot Price"},
    {**t, "y": balancing_price_values, "name": "Balancing Price"},
    {**t, "y": max_price_values, "name": "Max Price"}
]
go.Figure(traces)

In [105]:
traces = [
    {**t, "y": np.array(spot_price_values, dtype=float) - np.array(balancing_price_values, dtype=float), "name": "Spot minus Balancing Price"},
]
layout = {
    "yaxis": {"title": "Spot Price minus Balancing Price"},
    "title": "Data for 16:00"
}
go.Figure(data=traces, layout=layout)

In [106]:
max_prices.view("""select avg(spot_price), avg(balancing_price), avg(abs(spot_price - balancing_price)), avg(max_price) from _""")

|avg\(spot\_price\)|avg\(balancing\_price\)|avg\(abs\(spot\_price \- balancing\_price\)\)|avg\(max\_price\)|
|-|-|-|-|
|202\.41865971269488|198\.03634318986232|46\.43941145981484|223\.44720718118518|


In [107]:
[max_price_values] = zip(*max_prices.view("""select max_price from _ where max_price is not null"""))

In [108]:
import numpy as np

In [109]:
[hist, bin_edges] = np.histogram(max_price_values, bins=2**8)
cdf = np.cumsum(hist) / np.sum(hist)

In [110]:
traces = [
    {"x": bin_edges[:-1], "y": cdf}
]
go.Figure(traces)